In [2]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # 콘텍스트

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(ABC):  # 전략 : 추상 베이스 클래스

    @abstractmethod
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환한다."""


class FidelityPromo(Promotion):  # 첫 번째 구체적인 전략
    """충성도 포인트가 1,000점 이상인 고객은 전체 주문에 대해 5% 할인을 적용"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):  # 두 번째 구체적인 전략
    """하나의 주문에서 20개 이상의 동일 상품을 구입하면 해당 상품에 대해 10% 할인을 적용"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # 세 번째 구체적인 전략
    """서로 다른 상품을 10종류 이상 주문하면 전체 주문에 대해 7% 할인을 적용"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [3]:
# 충성도 점수. joe는 0점, ann은 1100점이다. 
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

# 한 쇼핑 카트의 3가지 항목
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]

# FidelityPromo 할인은 joe에게 아무런 할인이 없다
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [4]:
# ann은 충성도가 1000점이 넘으므로 5% 할인을 받는다.
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [6]:
banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]

# BulkItemPromo 할인 덕분에 joe는 바나나에 대해 1.50달러를 할인 받는다. 
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [7]:
# 각각 1달러인 10개의 서로 다른 상품이 있다. 
long_order = [LineItem(str(item_code), 1, 1.0)
             for item_code in range(10)]

# LargeOrderPromo 덕분에 전체 주문에 대해 7% 할인을 받는다 
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

### 6.1.2 함수 지향 전략

In [8]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # 콘텍스트

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            # 할인액을 계산하려면 self.promotion() 함수를 호출하면 된다.
            discount = self.promotion(self)  
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# 추상 클래스가 제거되었다. 

def fidelity_promo(order):  # 각각의 구체적인 전략이 함수로 구현되었다. 
    """충성도 포인트가 1,000점 이상인 고객은 전체 주문에 대해 5% 할인을 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """하나의 주문에서 20개 이상의 동일 상품을 구입하면 해당 상품에 대해 10% 할인을 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    """서로 다른 상품을 10종류 이상 주문하면 전체 주문에 대해 7% 할인을 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [9]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]

Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [10]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [11]:
banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]

Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [12]:
long_order = [LineItem(str(item_code), 1, 1.0)
             for item_code in range(10)]

Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [13]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

### 6.1.3 단순한 접근법

In [15]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos) # 제너레이터 사용

In [27]:
# 1. long_order 카트를 가지고 있으면 best_promo는 larger를 선택한다. 
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [28]:
# 2. banana_cart 를 가지고 있으면 bulk로 많이 할인받을 수 있다.
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [29]:
# 3. ann은 카트를 가지고만 있어도 fidelity로 많은 할인을 받을 수 있다. 
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

### 6.1.4 모듈에서 전략 찾기

In [26]:
# globals() 함수가 반환한 딕셔너리에서 name을 반복한다. 
promos = [globals()[name] for name in globals() 
         if name.endswith('_promo') 
         and name != 'best_promo'] # 무한 재귀를 피하기 위해 best는 생략

def best_promo(order):
    return max(promo(order) for promo in promos) 

## 6.2 명령

In [32]:
class MacroCommand:
    """명령 리스트를 실행하는 명령"""
    
    def __init__(self, commands):
        # commands 인수로부터 리스트를 만들면 명령들이 반복 가능한 객체임을 보장
        # 각각의 MacroCommand 객체 안에 명령에 대한 참조를 복사하게 된다. 
        self.commands = list(commands) 
        
    def __call__(self):
        # MacroCommand 객체가 호출되면 self.commans에 들어 있는 명령이 순서대로 호출된다. 
        for command in self.commands:
            command()